Setup Libraries


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections 
import re 
import unicodedata 
import six
import tensorflow as tf 

In [ ]:
def validate_case_matches_checkpoint(do_lower_case, init_checkpoint):
    """ Kiểm tra xem liệu cấu hình vỏ bọc có nhất quán với tên checkpoint .
    Đoạn mã này được sử dụng để kiểm tra xem cấu hình chữ hóa hay chữ thường do_lower_case có 
    phù hợp với tên checkpoint hay không . 

    Checkpoint là một tệp chứa các trọng số của mô hình bERY ĐÃ ĐƯỢC TIỀN HUẤN LUYỆN TRÊN DỮ LIỆU CỤ THỂ 
    """
    # người dùng phải chuyển vỏ bọc này vào và không có kiểm tra dõ dàng nào về việc 
    # liệu nó có khác với checkpoint không . Thông tin vỏ bọc đáng lẽ phải được lưu trữ trong 
    # tệp bert.cònig, nhưng nó không vì vậy phải phát hiện nó theo phương pháp phòng đoán và 
    # xác thực 



    # kiểm tra xem init_checkpoint có được khởi tạo hay không 
    if not init_checkpoint: 
        # lập tưc trả vè
        return 

    # Còn không thì sử dụng biểu thức chính qua regulary experession để tách tên mô hình từ init_checkpoint 
    # sử dụng hàm re.match để xem một chuỗi có khơp shay phù hợp với một biểu thức chính quy hay không 
    m = re.match("^.*?([A-Za-z0-9_-]+)/bert_model.ckpt", init_checkpoint)
    # kiểm tả xem m có chưa tên của mô hình từ init checkpoint 
    if m is None: 
        # lập tức trả về 
        return  
    
    # tạo biến model_name  = nhóm các phần tử có trong m 
    # với nhóm con đầu tiên 
    model_name = m.group(1)

    # tạo 2 danh sách  là lowe_cased và cased_model chứa tên của các checkpoint init 
    lower_models = [
      "uncased_L-24_H-1024_A-16", "uncased_L-12_H-768_A-12",
      "multilingual_L-12_H-768_A-12", "chinese_L-12_H-768_A-12"
    ]

    # với cased_model là dạn sách chứa tên của các mô hình với các văn abnr dạng nguyên bản 
    # gồm cả chữ in lẫn thường 
    cased_models = [
      "cased_L-12_H-768_A-12", "cased_L-24_H-1024_A-16",
      "multi_cased_L-12_H-768_A-12"
    ]

    # Khởi tạo 1 biến is_bad_config  = False sử dụng để kiểm tra xem cấu hình chữ hoa 
    # chữ thường (do_lowe_case) có phù hợp với tên checkpoint hay không 
    # nếu không biến này giá trị = True và đoạn mã sẽ sinh ra một lỗi 
    is_bad_config = False # và để đảm bảo rằng Bert được tinh chỉnh theo cachs tương thích với 
    # các mô hình tiền huấn luyện. 
    # kiểm tra xem nhóm các phần tử tên mô hình trong model name có giống với tên của các 
    # mô hình trong danh sách lower_models và không thuộc do_lowe_cased
    if model_name in lower_models and not do_lower_case:
        # ta gán is bad_ = true 
        is_bad_config = True
        # Bieens actual là giá trị hiện tại của mô hình tiền huấn luyện 
        # nếu giá trị không phù hợp sẽ gây lỗi 
        actual_flag = "False"
        # case_naem tên của cách mô hình bert được tiền huấn luyênh dựa trên checkpoint 
        case_naem = "lowercased"
        opposite_flag = "True"

    # Thực hiện kiểm tra tương tự  với danh sách case_model 
    if model_name in cased_models and do_lower_case:
        is_bad_config = True
        actual_flag = "True"
        case_name = "cased"
        opposite_flag = "False"

    # nếu is_bed là true sẽ ném một lỗi  VALUEerror 
    if is_bad_config:
        raise ValueError(
            "You passed in `--do_lower_case=%s` with `--init_checkpoint=%s`. "
            "However, `%s` seems to be a %s model, so you "
            "should pass in `--do_lower_case=%s` so that the fine-tuning matches "
            "how the model was pre-training. If this error is wrong, please "
            "just comment out this check." % (actual_flag, init_checkpoint,
                                            model_name, case_name, opposite_flag))

Xây dựng phương thức chuyển đổi văn bản thành dạng unicode 

In [ ]:
# Xây dựng phương thức chuyển đổi văn bản thành dạng unicode 
def convert_to_unicode(text):
    """ chuyển đổi văn bản thành dnagj unicode nếu chưa có giả sử đầu vào UTF-8"""
    # kiểm tra phiên bản python config 
    if six.PY3: # = True 
        # sử dụng hàm isinstance giá trị không thay đổi để kiểm tra xem 
        # text có dạng string hay không 
        if isinstance(text , str):
            return text 
        # trường hợp nếu text dạng bytes
        elif isinstance(text, bytes):
            # sử dụng mã hóa unicode để đưa dữ liệu về dạng unicode 
            # với decode("utf8" "ignore")
            return text.decode("utf8" ,"ignore")
        # trường hợp còn lại đưa ra cảnh báo lỗi 
        else: 
            raise ValueError("Unsuppoeted string type: %s" %(type(text)))
        
        # ta thực hiện tương tự với bản python 2
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")

# Xây dựng một phương thức tương tự mã hóa theo cách phù hợp để in hau tf.logging 
def printable_text(text):
  """Returns text encoded in a way suitable for print or `tf.logging`."""

  # These functions want `str` for both Python2 and Python3, but in one case
  # it's a Unicode string and in the other it's a byte string.
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text
    elif isinstance(text, unicode):
      return text.encode("utf-8")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")


In [ ]:
# Xây dựng phương thức load vocab_Size từ một từ điển 
def load_vocab(vocab_file):
    "Load a vocabulary file into a dictionary."
    # xây dựng một từ điển  với hàm OrderedDict là một lớp con củad dối tượng Dictionary 
    # nhưng khác ở nó duy trì thứu tự trèn của kháo . Nghãi là khi duyệt qua các phần tử của 
    # OrderedDict sẽ nhận được chúng theo thứ tự đã thêm vào không phải ngẫu nhiên như từ điển 
    vocab = collections.OrderedDict()
    # tạo một biến index init =  0 
    index = 0 
    # mở file vocab đặt cho phéo đọc file và gán nó vaò biến reader 
    with tf.gfile.GFile(vocab_file , "r") as reader: 
        # Trong khi điều kiện = True 
        while True : 
            # sử dụng biến token để luuư trữ các dòng mã được đọc từ các file reader 
            # đã được sử lý mã hóa unicode 
            token = convert_to_unicode(reader.readline())
            # nếu không có danh sách token nào được tạo ra 
            if not token : 
                # lập tức dừng lại 
                break 
            # chuẩn hóa danh sách toke với hàm strip 
            token = token.strip()
            # đặt các chỉ số token qua tập từ điển và đặt chỉ số index 
            # cho thứ tự các toke được đư vào từ điển
            vocab[token] = index 
            index += 1 
    # Trả về tập từ điển 
    return vocab 

# Xây dựng phuuwong thức chuyển đổi 1 chuỗi token[ids] bằng cách sử dụng bộ từ điển 
def convert_by_vocab(vocab, items):
    """ Convert a sequence of [tokens[ids]] using the vocab. 
        Chuyển đổi 1 chuỗi id mã thông báo bằng cách sử dụng tập từ điển 
    """
    # tạo một danh sách output = none để 
    output = []
    # duyệt qua dnah sách item là dnah sách chứa các tokens hoặc các id  cho tokens 
    for item in items: 
        # thêm các items vào bộ từ điển 
        output.append(vocab[item])
    # Trả về 1 danh sách output chứa các id hoặc các tokens tương ứng với các phần tử trong item
    return output 

# Xây dựng phuuwong thức  chuyển đổi các token thành dạng token_ids 
# nhận đầu vào là bộ từ điển chứa dnah sách các cặp (token , id) 
# và tokens là danh sách các tokens
def convert_tokens_to_ids(vocab, tokens):
    # trả về output chứa các id tương ứng với các tokens trong tokens 
    return convert_by_vocab(vocab , tokens)

# xây dựng phương thức chuyển đổi các id thành tokens 
# nhận đầu vào là bộ từ điển chưa các cặp toke , id và danh sách ids chưa các ids 
# sau đó trả về 1 danh sách chứa các tokens theo id tương ứng 
def convert_ids_to_tokens(inv_vocab, ids):
    return convert_by_vocab(inv_vocab, ids)


# Xây dựng phương thức dọn dẹp các khoảng trằng cơ bản 
def whitespace_tokenize(text):
    """ Chayj tinhs nanwg donj depj vaf phaan tachs canr khoangr trawngs 
    trong ddonj vawn banr .
    Run basic whitespace clearning and pslitting on a piece of text. 
    """
    text = text.strip()
    # kiểm tra xem có tồn tại text không nếu không trả về 1 dnah sách dỗng 
    if not text: 
        return []   
    # Trường hợp còn lại 
    tokens = text.split()
    return tokens 


In [ ]:
# Xây dựng lớp kiểu cha ký tự khoảng trắng 
def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically contorl characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  # gọi hàm unicodedata \.categorical để lấy một chuỗi mã hóa lại ký tự của char 
  cat = unicodedata.category(char)
  if cat == "Zs": # so sánh cat vs Zs một mã hóa cho ký tự khoảng trắng nếu = trả về True 
    # nếu char = " " => Zs
    return True
  return False

# kiểm tra xem liệu char có phải là một ký tự điều khiển 
def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat in ("Cc", "Cf"): # cc , cf là 2 mã hóa cho ký tự điều khiển 
    return True
  return False

# kiểm xem kiệu các ký tự có là ký tự đặc biệt 
def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  # So sánh cp với một số khoảng giá trị,
  #  tương ứng với các ký tự không phải là chữ cái hay số trong bảng mã ASCII1
  # Nếu cp nằm trong một trong các khoảng đó trả về True . 
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False

Xây dựn lớp Tokenize cơ bản 

In [1]:
# Xây dựng lớp tokenize cơ bản cho việc xử lý văn bản 
# đưa văn bản về dạng tiêu chuẩn 
class BasicTokenize(object):

    """Runs basic tokenizetion (punctuation splitting , lower casing , etc...)
    Chạy mã hóa mã thông báo cơ bản như xóa kỹ tự đặc biệt , biến đổi thường ... 
    """
    # Xây dựng phuuwong thức khởi tạo 
    def __init__(self, do_lower_case=True):
        """ Xây dựng một BasicTokenizer.
        Args : 
            do_lower_case: Whether to lower case the input.   
             Có viết thường chữ thường đầu vào hay không . 
        """

    # xây dựng phương thức mã hóa nhận đầu vào là text đoạn văn abnr 
    def tokenize(self, textr):
        """Tokenizes a piece of text."
            Token hóa 1 đoạn văn bản .
           """
        # Chuyển đổi văn abnr về dịnh dạng unicode 
        text = convert_to_unicode(text)
        # làm sạch văn bản bằng cách loại bỏ các ký tự không hợp lệ và khoảng trắng thừa 
        # sử dụng phương thức self.__clean_text của lớp 
        text = self._clean_text(text)
        # thêm khoảng trắng xung quanh các ký tự JCK sử dụng phương thức self.__tokenize_chines_
        text = self._tokenize_chinese_chars(text)

        # Tách text bằng các tokens dựa trên khoảng trắng giữ chúng 
        origi_tokens = whitespace_tokenize(text)
        # tạo một danh sách lưu trữ các tokens được tách 
        split_tokens = []
        # duyệt qua dnah sách các tokens đã được chuẩn hóa qua các lớp chuẩn háo ở trên 
        for token in origi_tokens: 
            # kiểm tra xem do_lower_case có bằng True không 
            if self.do_lower_case:
                # chuấn hóa tất cả dữ liệu về dạng thường 
                token = token.lower()
                # laoij bỏ dấu trọng âm có trong từ 
                token = self._run_strip_accents(token)
            # Loại bỏ các dấu câu . 
            split_tokens.extend(self._run_split_on_punc(token))

        # Sau đó thêm các khoẳng trắng cho đọa văn abnr 
        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens
    
    # Xây dựng phương thức loại bỏ đi các dấu từ nhận đầu vào là đoạn văn bản 
    def _run_strip_accents(self, text):
        """Strips accents from a piece of text. 
            Tách các dấu của từ trong đoạn văn bản . 
        """
        # sử dụng thư viện unicodedata để chuẩn háo văn bản theo dạng NFD (Normalization Form Decomposittion)
        # trong đó mỗi ký tự có dấu được phân tách thành một ký tự không dấu và một ký tự biể thị dấu 
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            #   Thêm các ký tự vào dnah sách output 
            output.append(char)
        return "".join(output)
    
    # Xây dựng phương thưcs loại bỏ dấu câu trong đoạn văn bản 
    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        # tạo một biến char lưu trữ dnah sách văn abnr 
        chars = list(text)
        # một biế  đếm i để đếm ác kỹ tự trong văn bản 
        i = 0
        # Tạo 1 biến start New_word  = True biến cho biết bắt đầu từ mới .  
        start_new_word = True
        # một dnah sách dỗng để lưu trữ các ký tự trong đoạn văn bản sau khi được loại bỏ dấu câu
        output = []
        # kiểm tra xem i là số lượng ký tự hiện tại có nhỏ hơn độ dài chars tring list 
        while i < len(chars):
            # gán một biến char = ký tự char tị chỉ số i 
            char = chars[i]
            # ,iểm tra xem biến char có phải dấu câu nếu phải 
            if _is_punctuation(char):
                # danh sách chứa char vào output
                output.append([char])
                # và đặt start_new_word bằng True.
                start_new_word = True
            else:
                # trường hợp còn lại ta kiểm tra xem star_new_word có = true 
                # nếu có ta thêm vào 1 phần tử = 1 khoảng không tồn tại 
                if start_new_word:
                    output.append([])
                    # đặt lại start)word  = false
                start_new_word = False
                # đặt chỉ số của ma trận output -1  = char thêm char vào danh sách cuối cùng 
                output[-1].append(char)
            # tăng i lên 1
            i += 1
        # trả về 1 dnah sách và thêm khoảng trắng vào cho nó . 
        return ["".join(x) for x in output]
    
    # xây dựng phương thức  kiểm tra xem CP là điểm mã của lý tự CJK không 
    def is_chinese_char(self, cp):
        """ Check whether CP is the codepoint of a CJK character."""
        # Điều này xác định một ký tượng tiếng trung như bất kỳ mọi thứ trong khối CJK Unicode. 
        
        # Sau đó kiểm tra xem điểm CP có nằm trong một trong các khoảng của khối CJK hay không ,
        # bằng cách so sánh cp với các gái trị hexa tương ứng 
        # Nếu CP nằm trong bất kỳ khoảng nào đoạn mã sẽ trả về True , => CP là một mã điểm của ký tự 
        if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
            (cp >= 0x3400 and cp <= 0x4DBF) or  #
            (cp >= 0x20000 and cp <= 0x2A6DF) or  #
            (cp >= 0x2A700 and cp <= 0x2B73F) or  #
            (cp >= 0x2B740 and cp <= 0x2B81F) or  #
            (cp >= 0x2B820 and cp <= 0x2CEAF) or
            (cp >= 0xF900 and cp <= 0xFAFF) or  #
            (cp >= 0x2F800 and cp <= 0x2FA1F)):  #

            return True
        # trường hợp còn lại False 
        return False 
    
    # xây dựng phương thức chuẩn hóa loại bỏ ký tự không hợp lệ hặc khoảng trắng 
    # trong text 
    def _clean_text(self, text):
        """Thực hiện loại bỏ các ký tự không hợp lệ và xóa bỏ các khoảng trắng trong văn bản."""
        # loại bỏ các ký tự không hợp lệ và các khoảng trắng thừa trong text
        output = []
        # duyệt qua đoạn văn bản 
        for char in text: 
            # sử dụng hàm ord để lấy điểm củ mỗi ký tự trong text và sử dụng hai hàm khác 
            # là _is_control và __is_whitesape để kiểm tra xem ký tự đó có phải là ký tự điều khiển 
            # hoặc khoảng trắng hay không . 
            cp = ord(char)
            # kiểm tra xem ký tự đó có phải ký tự điều khiển không 
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue 
            # thêm khoảng trắng vào văn bản . 
            if _is_whitespace(char):
                output.append(" ")
            
            else: 
                output.append(char)
        # Cuối cùng , hàm nối các phần tử trong danh sách output lại trả về một chuỗi ký tự mới    
        return "".join(output)


In [ ]:
# xây dựng lớp mã hóa Wordpiece 
class WordpieceTokenizer(object):
    """Runs WordPiece tokeniziation."""
    # thiết lập phương thức khởi tạo 
    def __init__(self, vocab , unk_token='[UNK]', max_input_chars_per_word = 200):
        self.vocab = vocab 
        self.unk_token = unk_token 
        self.max_input_chars_per_word = max_input_chars_per_word 

    def tokenize(self, text):
        """
        Tokenizes a piece of text ints its wword pieces. Token hóa một đoạn văn abnr thành các 
        phần từ của nó. 

        This uses a greedy longest-match-first algorithm to perform tokenization
        using the given vocabulary.

         Điều này sử dụng thuật toán khớp đầu tiên dài nhất để thực hiện mã thông báo bằng cách sử dụng 
         từ vựng đã cho . 

        For example : 
            input = "unaffable"
            output = ["un", "##aff", "##able"]

        ARGS: 
            text: A single token or whitespace separated tokens. This should have
            already been passed through `BasicTokenizer.
            
            Văn bản :Một mã thông báo đơn hoặc mã thông báo được phân tách thành khoảng trắng 
            điều này lẽ ra phải được thông qua BasicTokenizer. 

        Returns: 
            A list of wordpiece tokens. 
            Một dnah sách các thẻ từ . 

        """
        # chuyển tất acr về định dạng unicode 
        text = convert_to_unicode(text)

        # tạo một list output để lưu trữ danh sách đầu ra 
        output_tokens = []
        # duyệt qua 1 danh sách văn bản đã được loại bỏ đi các khoảng trắng. 
        for token in whitespace_tokenize(text):
            # tạo một list char  = danh sách các tokens 
            chars = list(token)
            # kiểm tra xem độ dài trong list char có lớn hơn maxtokens 
            if len(chars) > self.max_input_chars_per_word: 
                # Thêm ký tự phân cách [UNK] và vị trí cuối cùng của list 
                # và add nó vào danh sách lưu trữ 
                output_tokens.append(self.unk_token)
                continue
            
            # tạo 3 biến is_bad  , start = 0 
            is_bad = False # có ý nghĩa là có hay không mội chuỗi có trong chars mà không nămd trong tập từ điển 
            start = 0 
            sub_tokens = []
            
            # kiểm tra xem trong khi chỉ số start hiện tại có nhỏ hơn độ dài list chars 
            while start < len(chars):
                # đặt chỉ số kết thúc  = độ dài của list 
                end = len(chars)
                cur_substr = None 
                # xét chỉ số start hiện tại có nhỏ hơn chỉ số kết thúc hay không 
                while start < end: 
                    # thêm các khoẳng trắng vào trong list chars[start:end]
                    substr = "".join(chars[start:end])
                    # nếu start >0 thêm tiền tố ## vào substr 
                    if start > 0 :
                       substr = "##" + substr
                    # nếu substr thuộc bộ từ điển 
                    if substr in self.vocab: 
                        # ta gans cur_substr = substr 
                        cur_substr = substr
                        break 
                    # giamr chỉ số kết thúc đi 1 để thu hẹp đi khoăng cách giữa start và end 
                    # và tạo ra các chuỗi ngắn hơn từ chars mục đích tìm ra chuỗi con ngắn hơn 
                    # mà vẫn nằm trong từ điển self.vocab 
                    end -= 1  

                # nếu cur_sub vẫn  = None
                if cur_substr is None:
                    # gán is_bad = true và thoát 
                    is_bad = True
                    break
                # nếu không thêm thêm cur_sub vào dnah sách subtokens
                sub_tokens.append(cur_substr)
                # và đăth chỉ số start = end 
                start = end
            # nếu ibad = True
            if is_bad:
                # thì thêm một token không xác định self.unk_token vào danh sách output_tokens.
                output_tokens.append(self.unk_token)
            else: # thì mở rộng danh sách output_tokens bằng các phần tử trong sub_tokens.
                output_tokens.extend(sub_tokens)
        return output_tokens
       


In [ ]:
# xây dựng lớp Xử lý văn bản đầy đủ bằng việc kết hợp các phương thức đã xây dựng trươc đó 
class FullTokenizer(object):
  """Runs end-to-end tokenziation."""
  # Thiết lập phương thức khởi tạo và định dạng một số phương thức , lớp xử lý dữ liệu tiêu chuẩn 
  def __init__(self, vocab_file, do_lower_case=True):
    # Load data từ vocab_file gán nó vào biến vocab 
    self.vocab = load_vocab(vocab_file)
    # tạo một biến lưu trữ các phần tử trong từ điển đã được truy xuất 
    self.inv_vocab = {v: k for k, v in self.vocab.items()}
    # Mã hóa tiêu chuẩn văn bản cơ bản với bỏ khoảng trắng và các định dnagj cũng như ký tự đặc biệt 
    self.basic_tokenizer = BasicTokenize(do_lower_case=do_lower_case)
    # Mã háo tokens với WordpieceTokenizer 
    self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

  # Thiết lập lớp xử lý với các lớp đã được xây dựng trước đó 
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    # Trả về danh sách các tokens đã được trích xuất 
    return split_tokens

  # lấy ra IDS của theo chỉ số tokens 
  def convert_tokens_to_ids(self, tokens):
    return convert_by_vocab(self.vocab, tokens)

  # lấy  ra chỉ số tokens từ chỉ số ids 
  def convert_ids_to_tokens(self, ids):
    return convert_by_vocab(self.inv_vocab, ids)

